In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from oasis.plotting import simpleaxis
from oasis.oasis_methods import oasisAR1, oasisAR2
from oasis.functions import gen_data, gen_sinusoidal_data, deconvolve, estimate_parameters


import warnings
warnings.filterwarnings('ignore')

/Users/xiaoqiansun/opt/anaconda3/lib/python3.8/site-packages/oasis/functions.py:13: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  warn("Could not find cvxpy. Don't worry, you can still use OASIS, " +


In [2]:
# prep info
# contact: c
# status inlcude: r, s, a

root_path = r'/Users/xiaoqiansun/Desktop/MedLu/TubeTest/Data'


days = 6
fpsN = 15
fpsB = 30
baselinePeriod = 180 #(3min(180s) baseline)
redundentTime = 190 #10s+3min


M32_list = ['32-1_28', '32-2_18', '32-3_15', '32-4_20', '32-5_12', '32-6_15']
M33_list = ['33-1_28', '33-2_35', '33-3_32', '33-4_30', '33-5_30', '33-6_26']



# Define functions

In [3]:
# to keep certain status frrame interval
#----------------------------------------------------------------------------------
def pick_BInterval(m1_tube, m2_tube, s1, s2, c=None):      
      
    df1 = m1_tube.copy()
    df2 = m2_tube.copy()
    
    if c:
        df1 = df1[(df1['mouse1']== s1) & (df1['mouse2']== s2) & (df1['contact']== c)]
        df2 = df2[(df2['mouse1']== s1) & (df2['mouse2']== s2) & (df2['contact']== c)]
    
    else:
        df1 = df1[(df1['mouse1']== s1) & (df1['mouse2']== s2)]
        df2 = df2[(df2['mouse1']== s1) & (df2['mouse2']== s2)]
    
        
    df1 = df1.drop(['mouse1', 'mouse2', 'contact'], axis=1)
    df2 = df2.drop(['mouse1', 'mouse2', 'contact'], axis=1)
    
    return(df1, df2)




# calculate event rate&amplitude using oasis
#----------------------------------------------------------------------------------
def ER_AM_Oasis(df, fps):
        
    Frames, n_neuron = df.shape
    
    if Frames != 0 and n_neuron != 0:
        # get array
        df_array = df.to_numpy()

        spike_num = 0
        spike_mean = []
        spikeDFF_mean = []

        for i in range(n_neuron):
            y = df_array[:,i][~np.isnan(df_array[:,i])]
            c, s, b, g, lam = deconvolve(y, penalty=1)

            # pick non-zero spikes from deconvoluated trace
            sI = [i for i, e in enumerate(s) if e != 0]
            ss = [e for i, e in enumerate(s) if e != 0]

            # eliminate small spikes (only keep ones larger than non-zero mean)
            meanS = np.mean(ss)
            spikeIndex = [i for i, e in enumerate(ss) if e >= meanS]
            spikes = [e for i, e in enumerate(ss) if e >= meanS]

            spike_num = spike_num + len(spikeIndex)
            spike_mean.append(np.mean(spikes))
            spikeDFF_mean.append(np.mean(df_array[:,i][spikeIndex]))
        
        ER = spike_num/(Frames/fps)/n_neuron
        AM_spike = np.nanmean(spike_mean)
        AM_DFF = np.nanmean(spikeDFF_mean)
    else:
        ER = np.nan
        AM_spike = np.nan
        AM_DFF = np.nan
        
        
    return(ER, AM_spike, AM_DFF)



# Call

In [16]:
ER_32 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                      'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
ER_33 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                      'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
AM_32 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                      'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
AM_33 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                      'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
AM00_32 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                        'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
AM00_33 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                        'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})

Summary_s = [ER_32, AM_32, AM00_32, ER_33, AM_33, AM00_33]
Sheets_s = ['ER_32', 'AM_Spike_32', 'AM_DFF_32', 'ER_33', 'AM_Spike_33', 'AM_DFF_33']

for day in range(days):
    
    # read in dataframe
    m32_folder = os.path.join(root_path, M32_list[day])
    m33_folder = os.path.join(root_path, M33_list[day])
    
    
    m32_tube = pd.read_csv(os.path.join(m32_folder,'m32_tube.csv'), index_col = 'Frame')
    m33_tube = pd.read_csv(os.path.join(m33_folder,'m33_tube.csv'), index_col = 'Frame')     
    
    
    # separate sessions
    as1, as2 = pick_BInterval(m32_tube, m33_tube, 'a', 's', c=None)
    sa1, sa2 = pick_BInterval(m32_tube, m33_tube, 's', 'a', c=None)
    rs1, rs2 = pick_BInterval(m32_tube, m33_tube, 'r', 's', c=None)
    sr1, sr2 = pick_BInterval(m32_tube, m33_tube, 's', 'r', c=None)
    ar1, ar2 = pick_BInterval(m32_tube, m33_tube, 'a', 'r', c=None)
    ra1, ra2 = pick_BInterval(m32_tube, m33_tube, 'r', 'a', c=None)
    
    ss1, ss2 = pick_BInterval(m32_tube, m33_tube, 's', 's', c='nc')
    ssc1, ssc2 = pick_BInterval(m32_tube, m33_tube, 's', 's', c='c')
    
    # calculate ER, AM, AM00
    M32_dfs = [as1,sa1,rs1,sr1,ar1,ra1,ss1,ssc1]
    M33_dfs = [as2,sa2,rs2,sr2,ar2,ra2,ss2,ssc2]
    
    er_32s = []; am_32s = []; am00_32s = []
    er_33s = []; am_33s = []; am00_33s = []
    ss = [er_32s,am_32s,am00_32s,er_33s,am_33s,am00_33s]
    for i in range(len(M32_dfs)):
        
        er32, amS32, amF32 = ER_AM_Oasis(M32_dfs[i], fpsN)
        er33, amS33, amF33 = ER_AM_Oasis(M33_dfs[i], fpsN)
        er_32s.append(er32); am_32s.append(amS32); am00_32s.append(amF32)
        er_33s.append(er33); am_33s.append(amS33); am00_33s.append(amF33)
        
    
    # update into dataframe
    for m in range(len(Summary_s)):
        Summary_s[m].loc[day, ER_32.columns[1:]] = ss[m]
    
    
    
writer = pd.ExcelWriter('Summary_deconvoluatedSpikes.xlsx')
for k in range(len(Summary_s)):
    Summary_s[k].to_excel(writer, sheet_name=Sheets_s[k])



In [18]:
ERB_32 = pd.DataFrame({'Days':np.arange(1,7), 'ER':np.nan, 'AM_Spike': np.nan, 'AM_DFF':np.nan})
ERB_33 = pd.DataFrame({'Days':np.arange(1,7), 'ER':np.nan, 'AM_Spike': np.nan, 'AM_DFF':np.nan})

erB_32s = []; amB_32s = []; amDFFB_32s = []
erB_33s = []; amB_33s = []; amDFFB_33s = []

for day in range(days):
    
    m32_folder = os.path.join(root_path, M32_list[day])
    m33_folder = os.path.join(root_path, M33_list[day])
    
    
    m32_base = pd.read_csv(os.path.join(m32_folder,'m32_base.csv'), index_col = 'Frame')
    m33_base = pd.read_csv(os.path.join(m33_folder,'m33_base.csv'), index_col = 'Frame')
        
    er32, amS32, amF32 = ER_AM_Oasis(m32_base, fpsN)
    er33, amS33, amF33 = ER_AM_Oasis(m33_base, fpsN)
    
    erB_32s.append(er32); amB_32s.append(amS32); amDFFB_32s.append(amF32)
    erB_33s.append(er33); amB_33s.append(amS33); amDFFB_33s.append(amF33)
        
    
    for m in range(len(Summary_s)):
        Summary_s[m].loc[day, ER_32.columns[1:]] = ss[m]
    
    
ERB_32['ER'] = erB_32s; ERB_32['AM_Spike'] = amB_32s; ERB_32['AM_DFF'] = amDFFB_32s
ERB_33['ER'] = erB_33s; ERB_33['AM_Spike'] = amB_33s; ERB_33['AM_DFF'] = amDFFB_33s

ERB_32.to_excel(writer, sheet_name='Mouse 32 Baseline')
ERB_33.to_excel(writer, sheet_name='Mouse 33 Baseline')
writer.save()

# Plot

In [3]:
Sheets_32 = ['ER_32', 'AM_Spike_32', 'AM_DFF_32']
Sheets_33 = ['ER_33', 'AM_Spike_33', 'AM_DFF_33']


def plot_ER_DF(sheetIndex, bevavior, whichIndicator, compareOrNot=False):
    
    SummaryD1 = pd.read_excel('Summary_deconvoluatedSpikes.xlsx', sheet_name=Sheets_32[sheetIndex])
    SummaryD2 = pd.read_excel('Summary_deconvoluatedSpikes.xlsx', sheet_name=Sheets_33[sheetIndex])

    f, ax = plt.subplots(1,1, figsize=(10,6))
    ax.plot(SummaryD1[bevavior], '-.', marker='s', color='orange', label='Mouse 32 '+bevavior+' Deconvoluted')
    ax.plot(SummaryD2[bevavior], '-', marker='o', color='orange', label='Mouse 33 '+bevavior+' Deconvoluted')
    title = whichIndicator+'At' + bevavior + '_Deconvoluted'
    
    if compareOrNot:
        SummaryF1 = pd.read_excel('Summary.xlsx', sheet_name=Sheets_32[sheetIndex])
        SummaryF2 = pd.read_excel('Summary.xlsx', sheet_name=Sheets_33[sheetIndex])
        ax.plot(SummaryF1[bevavior], '-.', marker='s', color='blue', label='Mouse 32 '+bevavior+' DFF')
        ax.plot(SummaryF2[bevavior], '-', marker='o', color='blue', label='Mouse 33 '+bevavior+' DFF')
        title = whichIndicator+'At' + bevavior + '_DD'


    xDays = (SummaryD1['Days']-1).tolist()
    ax.set_xticks(xDays)
    
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    plt.legend(loc=1)
    plt.savefig(title)
    plt.close()

In [5]:
plot_ER_DF(0, 'rs', 'ER', compareOrNot=True)

# Test

In [ ]:
def plot_trace(groundtruth=False):
    plt.figure(figsize=(20,4))
    plt.subplot(211)
    plt.plot(b+c, lw=2, label='denoised')
    
    if groundtruth:
        plt.plot(true_b+true_c, c='r', label='truth', zorder=-11)
    plt.plot(y, label='data', zorder=-12, c='y')
    plt.legend(ncol=3, frameon=False, loc=(.02,.85))
    simpleaxis(plt.gca())
    
    
    plt.subplot(212)
    plt.plot(s, lw=2, label='deconvolved', c='g')
    
    if groundtruth:
        for k in np.where(true_s)[0]:
            plt.plot([k,k],[-.1,1], c='r', zorder=-11, clip_on=False)
    plt.ylim(0,1.3)
    plt.legend(ncol=3, frameon=False, loc=(.02,.85));
    simpleaxis(plt.gca())
    
    if groundtruth:
        print("Correlation of deconvolved activity  with ground truth ('spikes') : %.4f" % np.corrcoef(s,true_s)[0,1])
        print("Correlation of denoised fluorescence with ground truth ('calcium'): %.4f" % np.corrcoef(c,true_c)[0,1])

In [ ]:
# here we generate some simulated fluorescence data and plot it
true_b = 2
y, true_c, true_s = map(np.squeeze, gen_data(N=1, b=true_b, seed=0))
c, s, b, g, lam = deconvolve(y, penalty=1)


plot_trace(False)

# Test

In [ ]:
fps=15
df = as1
print(df.shape)

In [ ]:
Frames, n_neuron = df.shape

if Frames != 0 and n_neuron != 0:
    # get array
    df_array = df.to_numpy()

    spike_num = 0
    spike_mean = []
    spikeDFF_mean = []

    for i in range(n_neuron):
        y = df_array[:,i][~np.isnan(df_array[:,i])]
        c, s, b, g, lam = deconvolve(y, penalty=1)

        # pick non-zero spikes from deconvoluated trace
        sI = [i for i, e in enumerate(s) if e != 0]
        ss = [e for i, e in enumerate(s) if e != 0]

        # eliminate small spikes (only keep ones larger than non-zero mean)
        meanS = np.mean(ss)
        spikeIndex = [i for i, e in enumerate(ss) if e >= meanS]
        spikes = [e for i, e in enumerate(ss) if e >= meanS]
        spike_num = spike_num + len(spikeIndex)
        spike_mean.append(np.mean(spikes))
        spikeDFF_mean.append(np.mean(df_array[:,i][spikeIndex]))
        

    ER = spike_num/(Frames/fps)/n_neuron
    AM_spike = np.nanmean(spike_mean)
    AM_DFF = np.nanmean(spikeDFF_mean)
else:
    ER = np.nan
    AM_spike = np.nan
    AM_DFF = np.nan
    
print(ER, AM_spike, AM_DFF)

In [ ]:
ER, AM_spike, AM_DFF = ER_AM_Oasis(df, fps)
print(ER, AM_spike, AM_DFF)

In [ ]:
i=2
y = df_array[:,i][~np.isnan(df_array[:,i])]
c, s, b, g, lam = deconvolve(y, penalty=1)
plot_trace(False)

In [ ]:
len(y)